In [1]:
import torch

from modules.transformer import TransformerModule
from modules.mamba import MambaModule

from dataset import FrankaDataModule

In [2]:
checkpoint = "/work/tc064/tc064/s2567498/virtual-sensing/lightning_logs/transformer/version_9/checkpoints/epoch=1-step=787.ckpt"
hparams = "/work/tc064/tc064/s2567498/virtual-sensing/lightning_logs/transformer/version_9/hparams.yaml"

In [3]:
model = TransformerModule.load_from_checkpoint(
    checkpoint_path=checkpoint, hparams_file=hparams
)

In [15]:
dm = FrankaDataModule(
    "/work/tc064/tc064/s2567498/data-w-camera",
    batch_size=1,
    num_workers=0,
    data_portion=0.1,
    episode_length=200,
    use_cpu=True,
)
dm.setup()
data_loader = dm.val_dataloader()

Using 1048/10488 data episodes. Episode length: 200


In [16]:
# Get the first sample
first_batch = next(iter(data_loader))

In [17]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model.predict(first_batch)

torch.Size([1, 200, 80])
